<a href="https://colab.research.google.com/github/blawok/Facial_Keypoints/blob/master/xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np 
import pandas as pd 
import os
import missingno as msno 

from sklearn.model_selection import train_test_split 
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
%matplotlib inline 

checker = 0
for dirname, _, filenames in os.walk('drive/My Drive/facial_keypoints_data/'):
    for filename in filenames:
        if (os.path.join(dirname, filename) is not None): checker += 1
if checker == 4: print('All files are available')

All files are available


In [0]:
# test = pd.read_csv('drive/My Drive/facial_keypoints_data/test/test.csv')
# test.info()

# Preprocessing



### Creating X and y

In [0]:
training = pd.read_csv('drive/My Drive/facial_keypoints_data/training/training.csv')
training['Image'] = training['Image'].apply(lambda x: np.fromstring(x, sep=' '))
training = training.dropna()

# scale pixel values to [0, 1]
X = np.vstack(training['Image'].values) / 255.
X = X.astype(np.float32)

y = training[training.columns[:-1]].values
y = (y - 48) / 48  # scale target coordinates to [-1, 1]
X, y = shuffle(X, y, random_state=42)  # shuffle train data
y = y.astype(np.float32)

### Train and test split

Now we need to create train and test sets for all models

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=666)

# **1st Model - Random Forest / XGBoost**

In [0]:
import xgboost
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import cross_val_score,KFold, GridSearchCV
from sklearn.metrics import mean_absolute_error

In [16]:
xgb_model = xgboost.XGBRegressor()

xgb_model.fit(X_train, y_train)

[13:49:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [17]:
y_pred = xgb_model.predict(X_train)

predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = mean_absolute_error(y_train, predictions)
print(accuracy)

ValueError: ignored

In [14]:
# random forest implementation

parameters_for_testing = {
   'colsample_bytree':[0.4,0.6,0.8],
  #  'gamma':[0,0.03,0.1,0.3],
  #  'min_child_weight':[1.5,6,10],
  #  'learning_rate':[0.1,0.07],
  #  'max_depth':[3,5],
  #  'n_estimators':[10000],
  #  'reg_alpha':[1e-5, 1e-2,  0.75],
  #  'reg_lambda':[1e-5, 1e-2, 0.45],
  #  'subsample':[0.6,0.95]  
}

                    
xgb_model = xgboost.XGBRegressor(learning_rate =0.1, n_estimators=1000, max_depth=5,
    min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, scale_pos_weight=1, seed=27)

gsearch1 = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')
gsearch1.fit(X_train, y_train)
print (gsearch1.grid_scores_)
print('best params')
print (gsearch1.best_params_)
print('best score')
print (gsearch1.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:   18.9s
[Parallel(n_jobs=6)]: Done   2 tasks      | elapsed:   18.9s
[Parallel(n_jobs=6)]: Done   3 tasks      | elapsed:   18.9s
[Parallel(n_jobs=6)]: Done   4 tasks      | elapsed:   18.9s
[Parallel(n_jobs=6)]: Done   5 out of  12 | elapsed:   18.9s remaining:   26.5s
[Parallel(n_jobs=6)]: Done   7 out of  12 | elapsed:   18.9s remaining:   13.5s
[Parallel(n_jobs=6)]: Done   9 out of  12 | elapsed:   18.9s remaining:    6.3s


KeyboardInterrupt: ignored